In [1]:
import scipy.io as sio  
import matplotlib.pyplot as plt  
import numpy as np  
import h5py
from scipy.sparse import *
from scipy import *

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS

class MatrixFactorization:
    def __init__(self, maxIter=15, regParam=0.01, rank=10):
        self.maxIter = maxIter
        self.regParam = regParam
        self.rank = rank
        conf = SparkConf().setAppName("appName").setMaster("local[*]")
        conf.set("spark.driver.memory","8g")
        conf.set("spark.executor.memory","8g")
        self.spark = SparkContext(conf=conf)
                    
        print("New SparkSession started...")

    def change_parameter(self, regParam):
        self.regParam = regParam

    def matrix_factorization(self, train_lst):
        ratings = self.spark.parallelize(train_lst)
        print('create dataframe!')
        model = ALS.train(ratings, self.rank, seed=10, \
                          iterations=self.maxIter, \
                          lambda_=self.regParam)
        print("MF DONE")
        userFeatures = sorted(model.userFeatures().collect(), key=lambda d: d[0], reverse=False)
        productFeatures = sorted(model.productFeatures().collect(), key=lambda d: d[0], reverse=False)
        userProfile = {each[0]: each[1].tolist() for each in userFeatures}
        itemProfile = {each[0]: each[1].tolist() for each in productFeatures}
        
        
        return userProfile, itemProfile

    def end(self):
        self.spark.stop()
        print("SparkSession stopped.")

In [3]:
MF_result = {}

In [4]:
level = 4

feature=h5py.File('pseudo_matrix_' + str(level) + '.mat')  
chosen_train = feature['pseudo_matrix'][:]
chosen_train = csc_matrix(chosen_train).T
train_lst = []
for userID, PseudoItemID in zip(find(chosen_train)[0], find(chosen_train)[1]):
    train_lst.append([userID, PseudoItemID, chosen_train[userID, PseudoItemID]])
    
RMSE = []
MF = MatrixFactorization()
lambda_list = [0.005, 0.01,  0.02,  0.04,  0.08, 0.16,  0.32,  0.64,  1.28,  2.56, 5.12]

for plambda in lambda_list:
    
    MF.change_parameter(plambda)    
    user_profile, item_profile = MF.matrix_factorization(train_lst)
    
    P = np.dot(np.array(list(user_profile.values())), np.array(list(item_profile.values())).T)
    MF_result.setdefault(level, []).append(P)
#     feature=h5py.File('targetPos_' + str(level) + '.mat')  
#     targetPos = feature['targetPos'][:]
#     targetPos = list(map(int, targetPos.T[0]))
#     targetPos = [i-1 for i in targetPos] 
#     P_test = P[:, targetPos]
    
#     rating_matrix_csc = load_npz('ml-1m/sparse_matrix_ml-1m_selected.npz').tocsc()
#     feature=h5py.File('test_list.mat')  
#     test_list = feature['test_list'][:]
#     test_list = list(map(int, test_list.T[0]))
#     test_list = [i-1 for i in test_list] 
#     UI_matrix_test = rating_matrix_csc[:,test_list]
#     P_test = multiply(P_test, (UI_matrix_test!=0).toarray())
#     error = P_test - UI_matrix_test
#     rmse = (multiply(error, error).sum()/(error!=0).sum())**0.5
#     RMSE.append(rmse)
#     print(rmse)
# min_RMSE = min(RMSE)
MF.end()

KeyError: "Unable to open object (object 'pseudo_matrix' doesn't exist)"

In [25]:
min_RMSE#2

1.0475228027689734

In [29]:
min_RMSE

1.08369745051376